# Linear Regression

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score

from scipy.stats import ttest_ind
from src.utilities import VIF, levenes_test
from src.data_pipeline import data_clean

%matplotlib inline

plt.style.use('ggplot')

#### Load Dataset

I've already split the data into train and test sets.

In [ ]:
diamonds = pd.read_csv('../data/train.csv', index_col=0)

diamonds.info()

In [ ]:
diamonds.head()

#### Scatter Matrix

In [ ]:
plt.figure(figsize=(15,15));
sns.pairplot(diamonds, diag_kind='kde');
plt.title("Scatter Matrix");

## Univariate Analyses of Continuous Features

### `carat`

Intuitively, I expect `carat` to be have a strong relationship with `price`.

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='carat', y='price', data=diamonds, alpha=0.25);

Looks relatively linear. There appears to be a heteroskedastic relationship between carat and price. I'll fit a univariate linear regression on price and carat and test the residuals with a Breusch-Pagan test.

####  Testing for Heteroskedasticity

In [ ]:
y = diamonds['price']
X = sm.add_constant(diamonds['carat'])

lm_carat = sm.OLS(y, X)
lm_carat_fitted = lm_carat.fit()

lm_carat_fitted.summary()

In [ ]:
resids = lm_carat_fitted.resid

f_stat, p_val, alt = sm.stats.diagnostic.het_goldfeldquandt(y, X)

print("F-statistic:", f_stat)
print("p-value:", p_val)

We fail to reject the null hypothesis that our data is homoskedastic.

In [ ]:
pearson_resids = lm_carat_fitted.resid_pearson

plt.figure(figsize=(15,10)); # I like big plots and I cannot lie
plt.scatter(x=range(len(resids)), y=pearson_resids, alpha=0.2);
plt.title("Standardized Residual Plot");
plt.axhline(c='black', ls='-.');

Looks good, let's move on.

#### Normal Distribution of Error

In [ ]:
fig, ax = plt.subplots(figsize=(15,10));
sm.qqplot(pearson_resids, ax=ax, line='s');

Between this terrible looking Q-Q plot and the Jacques-Bera test, our error does not look normally distributed. Let's log transform this puppy.

#### Log-Transformation

In [ ]:
lny = np.log(y)

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='carat', y=lny, data=diamonds, alpha=0.25);

In [ ]:
lm_log_carat = sm.OLS(lny, X)
lm_log_carat_fitted = lm_log_carat.fit()

lm_log_carat_fitted.summary()

This looks a lot better, quadratic probably.

JB test is still significant (large sample size :\) but the stat is much lower. Let's look at residuals.

In [ ]:
log_pearson_resids = lm_log_carat_fitted.resid_pearson

plt.figure(figsize=(15,10)); # I like big plots and I cannot lie
plt.scatter(x=range(len(resids)), y=log_pearson_resids, alpha=0.2);
plt.title("Standardized Residual Plot");
plt.axhline(c='black', ls='-.');

In [ ]:
fig, ax = plt.subplots(figsize=(15,10));
sm.qqplot(log_pearson_resids, ax=ax, line='s');

Not perfect, but nothing is. Let's move on, it looks good enough. We will proceed using the natty log of price.

In [ ]:
diamonds['lnprice'] = np.log(diamonds['price'])

#### Autocorrelation

Recall that our Durbin-Watson test statistic was 1.992, so we're good there.

#### Conclusion
So our univariate analysis of carat and price shows we have an $R^2$ of 0.846, I guess we can call it a day! Just kidding. Let's check out our other features and see how a quadratic model stacks up.

### `carat2`

In [ ]:
diamonds['carat2'] = diamonds['carat'] ** 2
y = diamonds['lnprice']
X = diamonds[['carat','carat2']]

lm_carat2 = sm.OLS(y, X)
lm_carat2_fitted = lm_carat2.fit()

lm_carat2_fitted.summary()

In [ ]:
pearson_resids = lm_carat2_fitted.resid_pearson

plt.figure(figsize=(15,10)); # I like big plots and I cannot lie
plt.scatter(x=range(len(resids)), y=pearson_resids, alpha=0.2);
plt.title("Standardized Residual Plot");
plt.axhline(c='black', ls='-.');

In [ ]:
fig, ax = plt.subplots(figsize=(15,10));
sm.qqplot(pearson_resids, ax=ax, line='s');

In [ ]:
np.sqrt(lm_carat2_fitted.mse_resid)

### `x`, `y`, `z`, `depth`,  and `table`

These all have to do with the dimensions and shape of the diamond, and even if there is some good stuff going on here I would bet all I got that there's a bunch of multicollinearity (amongst themselves and `carat`) as well. Let's check it out anyway.

### `x`

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='x', y='lnprice', data=diamonds, alpha=0.25);

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds['x'])

lm_x = sm.OLS(y, X)
lm_x_fitted = lm_x.fit()

lm_x_fitted.summary()

#### Heteroskedasticity

In [ ]:
resids = lm_x_fitted.resid

f_stat, p_val, alt = sm.stats.diagnostic.het_goldfeldquandt(y, X)

print("F-statistic:", f_stat)
print("p-value:", p_val)

In [ ]:
pearson_resids = lm_x_fitted.resid_pearson

plt.figure(figsize=(15,10)); 
plt.scatter(x=range(len(resids)), y=pearson_resids, alpha=0.2);
plt.title("Standardized Residual Plot");
plt.axhline(c='black', ls='-.');

#### Normal Distribution of Error

In [ ]:
fig, ax = plt.subplots(figsize=(15,10));
sm.qqplot(pearson_resids, ax=ax, line='s');

Looks okay.

#### Autocorrelation

D-W stat at 1.996, so we are good to go.

### `y`

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='y', y='lnprice', data=diamonds, alpha=0.25);

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds['y'])

lm_y = sm.OLS(y, X)
lm_y_fitted = lm_y.fit()

lm_y_fitted.summary()

#### Heteroskedasticity

In [ ]:
resids = lm_y_fitted.resid

f_stat, p_val, alt = sm.stats.diagnostic.het_goldfeldquandt(y, X)

print("F-statistic:", f_stat)
print("p-value:", p_val)

Hmmm, doesn't look so good.

In [ ]:
pearson_resids = lm_y_fitted.resid_pearson

plt.figure(figsize=(15,10)); 
plt.scatter(x=range(len(resids)), y=pearson_resids, alpha=0.2);
plt.title("Standardized Residual Plot");
plt.axhline(c='black', ls='-.');

Looks like it's just a couple of outliers messing it up.

#### Normal Distribution of Error

In [ ]:
fig, ax = plt.subplots(figsize=(15,10));
sm.qqplot(pearson_resids, ax=ax, line='s');

Looks bad

#### Autocorrelation

D-W stat at 1.990, so we are good to go.

### `z`

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='z', y='lnprice', data=diamonds, alpha=0.25);

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds['z'])

lm_z = sm.OLS(y, X)
lm_z_fitted = lm_z.fit()

lm_z_fitted.summary()

#### Heteroskedasticity

In [ ]:
resids = lm_z_fitted.resid

f_stat, p_val, alt = sm.stats.diagnostic.het_goldfeldquandt(y, X)

print("F-statistic:", f_stat)
print("p-value:", p_val)

In [ ]:
pearson_resids = lm_z_fitted.resid_pearson

plt.figure(figsize=(15,10)); 
plt.scatter(x=range(len(resids)), y=pearson_resids, alpha=0.2);
plt.title("Standardized Residual Plot");
plt.axhline(c='black', ls='-.');

Looks like it's just a couple of outliers messing it up.

#### Normal Distribution of Error

In [ ]:
fig, ax = plt.subplots(figsize=(15,10));
sm.qqplot(pearson_resids, ax=ax, line='s');

Looks  fine

#### Autocorrelation

D-W stat at 1.994, so we are good to go.

### `depth`

This one is literally a function of `x`, `y`, and `z`.

<center>$depth = \displaystyle \frac{2z}{x + y}$</center>

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='depth', y='lnprice', data=diamonds, alpha=0.25);

Kinda looks like it goes straight up.

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds['depth'])

lm_depth = sm.OLS(y, X)
lm_depth_fitted = lm_depth.fit()

lm_depth_fitted.summary()

Yeah, hah smell ya later.

### `table`

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='table', y='lnprice', data=diamonds, alpha=0.25);

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds['table'])

lm_table = sm.OLS(y, X)
lm_table_fitted = lm_table.fit()

lm_table_fitted.summary()

In [ ]:
np.sqrt(lm_table_fitted.mse_resid)

### Takeaways

So our univariate analyses indicate that `carat`, `x`, `y`, and `z` are good features.

## Categorical Features

### `cut`

Note: values for this are `Fair`, `Good`, `Very Good`, `Premium`, and `Ideal`. Ideal being best, and Fair being worst.

In [ ]:
diamonds['cut'].unique()

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='carat', y='lnprice', data=diamonds, alpha=0.4, hue='cut', cmap='viridis');

In [ ]:
plt.figure(figsize=(15,10));
sns.boxplot(x='cut', y='lnprice', data=diamonds);

The variances look alright from the boxplots, but we should do a Levene's Test to be sure.

In [ ]:
levenes_test(diamonds['lnprice'], diamonds['cut'])

Variances are not homogenous, so I shouldn't add it to my model, and visual inspection of the boxplots and scatter plot indicate that it isn't a very impactful feature.

### `color`

In [ ]:
sorted(diamonds['color'].unique())

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='carat', y='lnprice', data=diamonds, alpha=0.4, hue='color', cmap='viridis');

In [ ]:
plt.figure(figsize=(15,10));
sns.boxplot(x='color', y='lnprice', data=diamonds, order=sorted(diamonds['color'].unique()));

The variances look alright from the boxplots

In [ ]:
levenes_test(diamonds['lnprice'], diamonds['color'])

So our test is significant indicating that `color` violates the assumption of homogeneity of variances, but based on the boxplot it does look like `color` could be a good indicator. Here I'm gonna make a judgement call and keep color, since large sample sizes just lead to low p-values. We have a fairly large data set (>40,000 observations), so p-values are going to be low even if the violations are negligible. 


I want to try combining categories. I'll run some pairwise t-tests first. Since I'll be doing multiple comparisons, I'll need to consider this when evaluating my p-vaklues. A bonferroni correction is appropriate here.

First I'll do pairwise comparisons of the color grades with the next worst color grade, so there are 6 comparisons there, and my significance level $\alpha = 0.05$ becomes $\displaystyle \frac{\alpha}{m}=\frac{0.05}{6}=0.008333$

In [ ]:
color_dict = {color: diamonds['lnprice'][diamonds['color'] == color] for color in sorted(diamonds['color'].unique())}

In [ ]:
ttest_ind(color_dict['D'], color_dict['E'], equal_var=False)

In [ ]:
ttest_ind(color_dict['E'], color_dict['F'], equal_var=False)

In [ ]:
ttest_ind(color_dict['F'], color_dict['G'], equal_var=False)

In [ ]:
ttest_ind(color_dict['G'], color_dict['H'], equal_var=False)

In [ ]:
ttest_ind(color_dict['H'], color_dict['I'], equal_var=False)

In [ ]:
ttest_ind(color_dict['I'], color_dict['J'], equal_var=False)

It looks like we can combine a few categories.

In [ ]:
diamonds['recolor'] = diamonds['color']

diamonds['recolor'].replace(to_replace='D', value='DE', inplace=True)
diamonds['recolor'].replace(to_replace='E', value='DE', inplace=True)
diamonds['recolor'].replace(to_replace='F', value='FG', inplace=True)
diamonds['recolor'].replace(to_replace='G', value='FG', inplace=True)

levenes_test(diamonds['lnprice'], diamonds['recolor'])

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='carat', y='lnprice', data=diamonds, alpha=0.4, hue='recolor', cmap='viridis');

In [ ]:
plt.figure(figsize=(15,10));
sns.boxplot(x='recolor', y='lnprice', data=diamonds, order=sorted(diamonds['recolor'].unique()));

### `clarity`

In [ ]:
diamonds['clarity'].unique()

In [ ]:
sorted_clarity = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [ ]:
plt.figure(figsize=(15,10));
sns.scatterplot(x='carat', y='lnprice', data=diamonds, alpha=0.4, hue='clarity', cmap='viridis');

In [ ]:
plt.figure(figsize=(15,10));
sns.boxplot(x='clarity', y='lnprice', data=diamonds, order=sorted_clarity);

Variances look bad here, and the worst clarity grades have many outliers on the low end, and the best clarity grade has many outliers on the high end. I'm just going to disregard this feature.

## Multivariate Analysis

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds[['carat', 'carat2', 'x', 'y', 'z']])

lm_multi = sm.OLS(y, X)
lm_multi_fitted = lm_multi.fit()

lm_multi_fitted.summary()

#### Multicollinearity

The condition number of 346 indicates multicollinearity. Intuitively, it makes sense that the weight (`carat`) and volume (`x`, `y`, `z`) would be correlated. We can use the Variance Inflation Factor to measure collinearity.

In [ ]:
VIF(X.drop(columns='const'))

So as expected,`x`, `y`, and `z` have ridiculous VIF's. Let's move on with just `carat` and `carat2`.

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(diamonds[['carat', 'carat2']])

lm_multi2 = sm.OLS(y, X)
lm_multi2_fitted = lm_multi2.fit()

lm_multi2_fitted.summary()

Recall that I want to add `recolor`.

In [ ]:
y = diamonds['lnprice']
X = sm.add_constant(pd.get_dummies(diamonds[['carat', 'carat2', 'recolor']], drop_first=True))

lm_multi_color = sm.OLS(y, X)
lm_multi_color_fitted = lm_multi_color.fit()

lm_multi_color_fitted.summary()

#### Cross-Validate with scikit-learn

In [ ]:
y = diamonds['lnprice']
X = pd.get_dummies(diamonds[['carat', 'carat2', 'recolor']], drop_first=True)

linear = LinearRegression(n_jobs=-1)

np.mean(cross_val_score(linear, X, y, n_jobs=-1, cv=10))

### Regularization

#### Ridge Regression

In [ ]:
ridge = Ridge()

param_grid = {'alpha': [0.001, 0.01, 0.25, 0.5, 1, 2, 3, 5, 10]}

cv = GridSearchCV(ridge, param_grid)
cv.fit(X, y)

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

In [ ]:
ridge = Ridge()

param_grid = {'alpha': np.linspace(1,3)}

cv = GridSearchCV(ridge, param_grid, n_jobs=-1)
cv.fit(X, y)

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

In [ ]:
ridge = Ridge()

param_grid = {'alpha': np.linspace(2.25,2.27)}

cv = GridSearchCV(ridge, param_grid, n_jobs=-1)
cv.fit(X, y)

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

#### Lasso

In [ ]:
lasso = Lasso()

param_grid = {'alpha': np.linspace(0.001,10)}

cv = GridSearchCV(lasso, param_grid, n_jobs=-1)
cv.fit(X, y)

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

Marginal improvement with Ridge Regression, let's just move forward without any regularization.

### Test Score

In [ ]:
test = data_clean(pd.read_csv('../data/test.csv', index_col=0))

y_train = diamonds['lnprice']
X_train = pd.get_dummies(diamonds[['carat', 'carat2', 'recolor']], drop_first=True)

y_test = test['lnprice']
X_test = pd.get_dummies(test[['carat', 'carat2', 'recolor']], drop_first=True)

linear = LinearRegression(n_jobs=-1)
linear.fit(X_train,y_train)
linear.score(X_test,y_test)

In [ ]:
lm_final = sm.OLS(y_train, sm.add_constant(X_train))
lm_final = lm_final.fit()

lm_final.summary()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x='carat', y='lnprice', data=diamonds, alpha=0.4, hue='recolor', cmap='viridis');
sns.regplot(x='carat', y=)